In [ ]:
import pandas as pd
import os
import sys
import boto3
import io
import geopandas as gpd
import dask_geopandas 

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_gpkg_from_directory, upload_csv_aws
from scripts.utils.write_metadata import append_metadata

In [ ]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2b_reproject/governance/natural_resource_conservation/ca_state_water_resources_board/'

pull_gpkg_from_directory(bucket_name, aws_dir)

In [ ]:
# large file, so we break it up into partitions
water_well_data = dask_geopandas.read_file('GAMA_division_drinking_water_reproject.gpkg',npartitions=5)
# let's load in one partition so we can look at the data (up to about the first 1/5 of the rows)
df = water_well_data.partitions[0].compute()
df

In [ ]:
# lots of columns! Can we get rid of some to save memory?
df.columns

In [ ]:
# example of removing columns
# let's say these are the only ones we need to calculate our metrics
subset = water_well_data[[
    'gm_well_id',
    'gm_result_modifier',
    'gm_result',
    'gm_chemical_units', 
    'gm_reporting_limit',
    'GEOID'
]]

In [ ]:
# now that the dataframe is much smaller, we can load it in
subset_df = subset.compute()
subset_df